In [1]:
pip install PyPDF2

  Using cached pypdf2-3.0.1-py3-none-any.whl.metadata (6.8 kB)
Using cached pypdf2-3.0.1-py3-none-any.whl (232 kB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
from google.cloud import storage
from PyPDF2 import PdfReader, PdfWriter
from io import BytesIO 

In [3]:
bucket_name = ""
source_blob_name = "filename.pdf"
destination_folder_name = source_blob_name

In [4]:
storage_client = storage.Client()
bucket = storage_client.bucket(bucket_name)

# Download the source PDF from GCS
source_blob = bucket.blob(source_blob_name)
pdf_content = source_blob.download_as_bytes()

In [5]:
def split_pdf_in_gcs(bucket_name, source_blob_name, destination_folder_name):
    with BytesIO(pdf_content) as pdf_file:
        pdf_reader = PdfReader(pdf_file)
        num_pages = len(pdf_reader.pages)

        # Split and upload each page as a new PDF
        for page_num in range(num_pages):
            pdf_writer = PdfWriter()
            pdf_writer.add_page(pdf_reader.pages[page_num])

            # Create a unique name for each page PDF (adjust as needed)
            page_blob_name = f"{destination_folder_name}_page_{page_num + 1}.pdf"

            # Upload the page PDF to GCS
            page_blob = bucket.blob(page_blob_name)

            with BytesIO() as output_buffer:
                pdf_writer.write(output_buffer)  
                output_buffer.seek(0)
                page_blob.upload_from_file(output_buffer, content_type='application/pdf')
                pass

            print(f"Uploaded page {page_num + 1} to GCS: {page_blob_name}")

In [6]:
split_pdf_in_gcs(bucket_name, source_blob_name, destination_folder_name)

Uploaded page 1 to GCS: scratch/finances_lgb/lgbaeza VISA estado-de-cuenta-24-06-2024 (1).pdf_page_1.pdf
Uploaded page 2 to GCS: scratch/finances_lgb/lgbaeza VISA estado-de-cuenta-24-06-2024 (1).pdf_page_2.pdf
Uploaded page 3 to GCS: scratch/finances_lgb/lgbaeza VISA estado-de-cuenta-24-06-2024 (1).pdf_page_3.pdf
Uploaded page 4 to GCS: scratch/finances_lgb/lgbaeza VISA estado-de-cuenta-24-06-2024 (1).pdf_page_4.pdf
Uploaded page 5 to GCS: scratch/finances_lgb/lgbaeza VISA estado-de-cuenta-24-06-2024 (1).pdf_page_5.pdf
